In [14]:
import pandas as pd
import numpy as np
from datetime import datetime

In [15]:
df = pd.read_csv("fiis.csv", index_col=0)

df["data_pagamento"] = pd.to_datetime(df["data_pagamento"])

df["cotacao"] = df["cotacao"].str.replace("R$ ", "", regex=False)
df["cotacao"] = df["cotacao"].str.replace(".", "", regex=False)
df["cotacao"] = df["cotacao"].str.replace(",", ".", regex=False).astype(float)

df["rendimento"] = df["rendimento"].str.replace("R$ ", "", regex=False)
df["rendimento"] = df["rendimento"].str.replace(".", "", regex=False)
df["rendimento"] = df["rendimento"].str.replace(",", ".", regex=False).astype(float)

C:\Users\DDMX\AppData\Local\Temp\ipykernel_11564\389494241.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["data_pagamento"] = pd.to_datetime(df["data_pagamento"])


In [16]:
fii_tracked = "MCCI11"
initial_application = 100000
application_date = pd.to_datetime("01/01/2020")

fii_df = df[
    (df["cotacao"] != 0) &
    (df["data_pagamento"] >= application_date) &
    (df["fundo"] == fii_tracked)
]

starting_date = fii_df.iloc[-1]

num_cotas = np.floor(initial_application / starting_date["cotacao"])
rest = np.round((initial_application % starting_date["cotacao"]), 2)

print(starting_date)
print(num_cotas)
print(rest)



fundo                          MCCI11
data_pagamento    2020-04-20 00:00:00
cotacao                         87.45
dy                              0,57%
rendimento                        0.5
Name: 59, dtype: object
1143.0
44.65


In [17]:
fii_df = fii_df.reindex(index=fii_df.index[::-1]).iloc[1:]

In [18]:
fii_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 58 to 0
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fundo           59 non-null     object        
 1   data_pagamento  59 non-null     datetime64[ns]
 2   cotacao         59 non-null     float64       
 3   dy              59 non-null     object        
 4   rendimento      59 non-null     float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 2.8+ KB


In [19]:
acc = rest
total_dividends_period = 0
monthly_reinvestment = 1000

for i in range(len(fii_df)):
    line = fii_df.iloc[i]
    month_income = (num_cotas * line["rendimento"])
    
    total_dividends_period += month_income
    
    acc = monthly_reinvestment + month_income + acc

    if acc / line["cotacao"] > 0:
        num_papers = np.floor(acc / line["cotacao"])
        num_cotas += num_papers

        acc = np.round(acc % line["cotacao"], 2)
    else:
        acc += month_income



    # print(month_income)
print(f"Total de dividendos ganhos no período (até data do ultimo pagamento): {np.round(total_dividends_period, 2)}")

Total de dividendos ganhos no período (até data do ultimo pagamento): 99856.95
